### CREDIT RISK MODELING: APPLICATION OF DEEP LEARNING

In [2]:
from collections import Counter

**Introduction**

- The aim of this project is to use classical machine learning algorithms on a refined dataset and discuss the steps used to predict mortgage default. 

**Mortgage Delinquency**